In [8]:
import numpy as np
import cv2

def gammaCorrection(src, gamma):
    invGamma = 1 / gamma
    table = [((i / 255) ** invGamma) * 255 for i in range(256)]
    table = np.array(table, np.uint8)
    return cv2.LUT(src, table)

def get_binary_image(frame):
    enhanced_frame = gammaCorrection(frame, 2)
    gray_frame = cv2.GaussianBlur(cv2.cvtColor(enhanced_frame,
                                               cv2.COLOR_RGB2GRAY),(5,5),0)
    threshold = cv2.threshold(gray_frame, 200,255, cv2.THRESH_BINARY)[1]
    return (enhanced_frame, threshold)

def display_tracking_window_axis_center(frame,x,y,tw_width,tw_height, rows):
        bounding_box = cv2.rectangle(frame,
                  (x, y),
                  (x + tw_width, y + tw_height),
                  (255, 0, 0), 1)
        horisontal_axis = cv2.line(frame,
                        (x + int(tw_width/2), 0),
                        (x + int(tw_width/2), rows),
                        (255, 0, 0), 1)
        object_center = cv2.circle(frame,
                                (x + int(tw_width/2), y + int(tw_height/2)),
                                2, (0,0,255),2)
        
        return (bounding_box, horisontal_axis,object_center)

In [9]:
video = cv2.VideoCapture(r'.\input_data\2023_03_14_PTM.avi')

roi_start_x = 315
roi_end_x = 370
roi_start_y = 217
roi_end_y = 400

tw_start_point_x = 30
tw_start_point_y = 0
tw_start_width = 25
tw_start_height = 60
term_criteria= (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10,1)

while True:
    full_frame = video.read()[1]
    frame = full_frame[roi_start_x:roi_end_x, roi_start_y:roi_end_y]  
    threshold = get_binary_image(frame)[1]
    track_window = cv2.meanShift(threshold,
                                 (tw_start_point_x, tw_start_point_y,
                                  tw_start_width, tw_start_height),
                                  term_criteria)[1]
    tw_start_point_x, tw_start_point_y, tw_width, tw_height = track_window
    rows, cols, _bands = frame.shape

    bounding_box, horisontal_axis, object_center = display_tracking_window_axis_center(
        frame,tw_start_point_x, tw_start_point_y,tw_width,tw_height,rows)

    cv2.imshow('Frame', frame)
    key = cv2.waitKey(100) #for framerate 20, normal playback is 100
    if key == 27:
        break
video.release()
cv2.destroyAllWindows()